# **# 3.**

생선의 종을 분류하는 모델의 C값이 적절한 값인지, 기존의 코드에서 GridSearchCV를 추가하여 C값이 변화하였는지 확인해보기.

In [ ]:
import pandas as pd

fish = pd.read_csv('https://bit.ly/fish_csv_data')

fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()
fish_target = fish['Species'].to_numpy()

#data split
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42)

#preprocessing
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

#LR
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)

print('GridSearchCV 적용 전')
print('train score :',lr.score(train_scaled, train_target))
print('test score :',lr.score(test_scaled, test_target))
print('C = 20')

GridSearchCV 적용 전
train score : 0.9327731092436975
test score : 0.925
C = 20


In [ ]:
#GridSearchCV
from sklearn.model_selection import GridSearchCV
params = {'C' : [i for i in range(1, 30)]}
gs = GridSearchCV(LogisticRegression(max_iter=1000), params, n_jobs=-1)
gs.fit(train_scaled, train_target)
dt = gs.best_estimator_

print('GridSearchCV 적용 후')
print('train score :',dt.score(train_scaled,train_target))
print('test score :',dt.score(test_scaled,test_target))
print('C =',gs.best_params_['C'])

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  UserWarning,


GridSearchCV 적용 후
train score : 0.9159663865546218
test score : 0.925
C = 11


# **# 5.**

행성의 측정 방법
(Grid Search 사용 X)

In [ ]:
# 데이터 불러오기
from sklearn import datasets
import pandas as pd
import seaborn as sns
planets=sns.load_dataset('planets')
print(planets.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   method          1035 non-null   object 
 1   number          1035 non-null   int64  
 2   orbital_period  992 non-null    float64
 3   mass            513 non-null    float64
 4   distance        808 non-null    float64
 5   year            1035 non-null   int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 48.6+ KB
None


In [ ]:
# 결측치 채우기############악!
print(planets.groupby('method').mean())

planets["orbital_period"] = planets.groupby("method").transform(lambda x: x.fillna(x.mean()))
planets["distance"] = planets.groupby("method").transform(lambda x: x.fillna(x.mean()))

print(planets.info())

                                 number  orbital_period      mass  \
method                                                              
Astrometry                     1.000000      631.180000       NaN   
Eclipse Timing Variations      1.666667     4751.644444  5.125000   
Imaging                        1.315789   118247.737500       NaN   
Microlensing                   1.173913     3153.571429       NaN   
Orbital Brightness Modulation  1.666667        0.709307       NaN   
Pulsar Timing                  2.200000     7343.021201       NaN   
Pulsation Timing Variations    1.000000     1170.000000       NaN   
Radial Velocity                1.721519      823.354680  2.630699   
Transit                        1.954660       21.102073  1.470000   
Transit Timing Variations      2.250000       79.783500       NaN   

                                  distance         year  
method                                                   
Astrometry                       17.875000  2011.500000

ValueError: ignored

In [ ]:
# 데이터프레임의 특성 나누기
data = planets[['number','orbital_period','distance','year']].to_numpy()
target = planets['method'].to_numpy()

# 훈련 세트와 테스트 세트로 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data,target,test_size=0.2,random_state=42)

# 표준화 전처리를 하지 않은 결정 트리 모델
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(train_input,train_target)
print(dt.score(train_input,train_target))
print(dt.score(test_input,test_target))

# 트리 그래프 그리기
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
plt.figure(figsize=(20,15))
plot_tree(dt,filled=True,feature_names=['number','orbital_period','distance','year'])
plt.show()